This code's methods is borrowed from yichen's code in his folder. The purpose of this file is to clean the income csv file retrieved from the us census website and merge with our eviction dataset.

Done by: Merna Alghannam, Team 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
ROOT_FOLDER = "/Users/merna/OneDrive/Desktop/ds-state-ma-housing-comitt/"

In [7]:
#Merge evictions data with household_normalized file
household_new = pd.read_csv(ROOT_FOLDER + "Eviction_Cases/household_normalization.csv",index_col=0)
household_new = household_new.sort_values('Evictions per Rented Households', ascending=False)


#income per household per municipality
income = pd.read_csv(ROOT_FOLDER+'Muni_Profile/2019_5Y_Income.csv',skiprows=1)
#occupation of people in municipalities
cdf = pd.read_csv(ROOT_FOLDER+'Eviction_Cases/eviction_num_by_muni.csv')

counts = cdf[cdf['Muni'].isin(household_new.Muni)]

# #data cleaning function 1
def df_clean(df):
    muni_list = []
    for x in range(df.shape[0]):
        wsplit = []
        wsplit = df.iloc[x]['Geographic Area Name'].split()
        word = ' '
        reject_list = ['town','Town','city','City','city,','City,','town,','Town,']
        for w in wsplit:
            if w in reject_list:
                break
            else:    
                word += ' ' + w 
        muni_list.append(word)    
    df['Geographic Area Name'] = muni_list
    df['Geographic Area Name'] = df['Geographic Area Name'].str.lstrip()
    df['Geographic Area Name'] = df['Geographic Area Name'].str.rstrip()    
    
    return df

#data cleaning function 2
def keep_estimates(idf):
    boolCols = idf.columns.str.contains('Estimate')
    newCols = list(idf.columns[0:2])
    for col in range(2,len(boolCols)):
        if boolCols[col] == True:
            newCols.append(idf.columns[col])
    df = idf[newCols]
    
    return df

income = df_clean(keep_estimates(income));

#Merging munis with corresponding socio-economic features
income_df =  pd.merge(counts,income,left_on='Muni',right_on='Geographic Area Name').drop(columns=['id','Geographic Area Name'])
income_with_rate = pd.merge(household_new, income_df, left_on=household_new['Muni'], right_on=income_df.Muni)
income_with_rate = income_with_rate.drop(['Muni_x', 'Muni_y', 'Eviction Numbers_y'], axis=1)
income_with_rate.rename(columns={'key_0':'Muni', ' Eviction Numbers_x': 'eviction_num'}, inplace=True)

income_with_rate['Median Income'] = income_with_rate['Estimate!!Households!!Median income (dollars)'].astype(str).replace('250,000+', '250000').astype(float)
income_with_rate['Rent Burden'] = income_with_rate['Estimate!!Households!!PERCENT ALLOCATED!!Household income in the past 12 months'].astype(float)

#only take those columns I wanted
#which are those listed below
income_with_rate = income_with_rate[["Muni", "Eviction Numbers_x", "Renter Occupied", "Evictions per Rented Households",  'Median Income', 'Rent Burden', "Renter Occupied Pct"]]

#save cleaned income code merged with household_normalize
income_with_rate.to_csv(ROOT_FOLDER+"Eviction_Cases\income_normalized.csv")

C:\Users\merna\anaconda3\envs\tf1.13\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\merna\anaconda3\envs\tf1.13\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\merna\anaconda3\envs\tf1.13\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta